In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define a residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # If the input and output dimensions don't match, use a 1x1 convolution to adjust
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(residual)  # Add the residual connection
        out = self.relu(out)
        return out

# Define the ResNet-like model
class ResNetMNIST(nn.Module):
    def __init__(self):
        super(ResNetMNIST, self).__init__()
        self.in_channels = 16  # Initial number of channels

        # Initial convolution layer
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        # Residual blocks
        self.layer1 = self.make_layer(16, 2, stride=1)
        self.layer2 = self.make_layer(32, 2, stride=2)
        self.layer3 = self.make_layer(64, 2, stride=2)

        # Fully connected layer
        self.fc = nn.Linear(64, 10)

    def make_layer(self, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)  # Only the first block uses the specified stride
        layers = []
        for stride in strides:
            layers.append(ResidualBlock(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)

        # Global average pooling
        out = nn.functional.avg_pool2d(out, out.size()[2:])
        out = out.view(out.size(0), -1)  # Flatten
        out = self.fc(out)
        return out

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize with mean and std of MNIST
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetMNIST().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}")

# Testing loop
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.2f}%)")

# Train and test the model
for epoch in range(1, 11):  # 10 epochs
    train(epoch)
    test()

Train Epoch: 1 [0/60000] Loss: 2.439350
Train Epoch: 1 [6400/60000] Loss: 0.204953
Train Epoch: 1 [12800/60000] Loss: 0.134977
Train Epoch: 1 [19200/60000] Loss: 0.072310
Train Epoch: 1 [25600/60000] Loss: 0.068027
Train Epoch: 1 [32000/60000] Loss: 0.091741
Train Epoch: 1 [38400/60000] Loss: 0.032694
Train Epoch: 1 [44800/60000] Loss: 0.023565
Train Epoch: 1 [51200/60000] Loss: 0.040834
Train Epoch: 1 [57600/60000] Loss: 0.099202
Test set: Average loss: 0.0016, Accuracy: 9667/10000 (96.67%)
Train Epoch: 2 [0/60000] Loss: 0.019852
Train Epoch: 2 [6400/60000] Loss: 0.034285
Train Epoch: 2 [12800/60000] Loss: 0.104828
Train Epoch: 2 [19200/60000] Loss: 0.021167
Train Epoch: 2 [25600/60000] Loss: 0.064488
Train Epoch: 2 [32000/60000] Loss: 0.006911
Train Epoch: 2 [38400/60000] Loss: 0.132795
Train Epoch: 2 [44800/60000] Loss: 0.059097
Train Epoch: 2 [51200/60000] Loss: 0.069208
Train Epoch: 2 [57600/60000] Loss: 0.028235
Test set: Average loss: 0.0008, Accuracy: 9868/10000 (98.68%)
Train 